## ASPLOS'21 MASA: Matching by Research Area + Random

In [ ]:
debug = False
import pandas
import numpy as np
from IPython.display import display, HTML
pandas.set_option('display.max_columns', None)  # or 1000
pandas.set_option('display.max_rows', None)  # or 1000
#pandas.set_option('display.max_colwidth', None)  # or 199


## Datafiles
mentors_datafile = "mentors_final_uploaded.csv" 

## Read Dataframe
mentors_df = pandas.read_csv(mentors_datafile) 
mentors_df = mentors_df.replace(np.nan, '', regex=True)

## Filter Mentors With Alllocated Mentees 
mentors_df = mentors_df[mentors_df["MenteeAllocated"] == True]
#display(mentors_df)

## Merge Multiple Entries for A Single Mentor
mentors_dedup_df = mentors_df
mentors_dedup_groupby = mentors_dedup_df.groupby(['FirstName','LastName','Email',
                                                  'Affiliation', 'SeniorResearcher/Student',
                                                  'MentorSignup','NumMentees','AffiliationType',
                                                  'ResearchArea','MenteeSignup'])
mentors_dedup_menteename  = mentors_dedup_groupby['MenteeName'].apply(';'.join).reset_index()
mentors_dedup_menteeemail = mentors_dedup_groupby['MenteeEmail'].apply(';'.join).reset_index()
mentors_dedup_menteeaff   = mentors_dedup_groupby['MenteeAffiliation'].apply(';'.join).reset_index()

## Merge all three dedup dataframes into single dataframe.
mentors_dedup_df = pandas.concat([mentors_dedup_menteename, mentors_dedup_menteeemail['MenteeEmail'],
                              mentors_dedup_menteeaff['MenteeAffiliation']], axis=1)

## Split MenteeName, Email, Aff into separate columns (1 per mentee)   
mentors_dedup_df[["MenteeName1", "MenteeName2", "MenteeName3"]] = mentors_dedup_df["MenteeName"].str.split(pat=";", expand=True)
mentors_dedup_df[["MenteeEmail1", "MenteeEmail2", "MenteeEmail3"]] = mentors_dedup_df["MenteeEmail"].str.split(pat=";", expand=True)
mentors_dedup_df[["MenteeAffiliation1", "MenteeAffiliation2", "MenteeAffiliation3"]] = mentors_dedup_df["MenteeAffiliation"].str.split(pat=";", expand=True)

mentors_dedup_df = mentors_dedup_df[['FirstName','LastName','Email',
                                    'Affiliation', 'SeniorResearcher/Student',
                                    'MentorSignup','NumMentees','AffiliationType',
                                    'ResearchArea','MenteeSignup',
                                    "MenteeName1","MenteeEmail1","MenteeAffiliation1",
                                    "MenteeName2","MenteeEmail2","MenteeAffiliation2",
                                    "MenteeName3","MenteeEmail3","MenteeAffiliation3"]]

## Print Sanity Check
print("Original Mentee Count: "+str((~mentors_dedup_df['MenteeEmail1'].isnull()).sum() + (~mentors_dedup_df['MenteeEmail2'].isnull()).sum() + (~mentors_dedup_df['MenteeEmail3'].isnull()).sum())) 
print("Deduplicated Mentee Count: "+ str((~mentors_df['MenteeEmail'].isnull()).sum()))

## Split 1-Mentee, 2-Mentee and 3-Mentee
mentors_dedup_1mentee_df = mentors_dedup_df[~mentors_dedup_df['MenteeEmail1'].isnull() &
                                            mentors_dedup_df['MenteeEmail2'].isnull()  &
                                            mentors_dedup_df['MenteeEmail3'].isnull()]
mentors_dedup_2mentee_df = mentors_dedup_df[~mentors_dedup_df['MenteeEmail1'].isnull() &
                                            ~mentors_dedup_df['MenteeEmail2'].isnull() &
                                            mentors_dedup_df['MenteeEmail3'].isnull()]
mentors_dedup_3mentee_df = mentors_dedup_df[~mentors_dedup_df['MenteeEmail1'].isnull() &
                                            ~mentors_dedup_df['MenteeEmail2'].isnull() &
                                            ~mentors_dedup_df['MenteeEmail3'].isnull()]

## Dump csv files for mentors_replicated_df and mentees_df 
mentors_dedup_1mentee_df.to_csv('mailmerge_masa_mentors1.csv', index=False)
mentors_dedup_2mentee_df.to_csv('mailmerge_masa_mentors2.csv', index=False)
mentors_dedup_3mentee_df.to_csv('mailmerge_masa_mentors3.csv', index=False)

#display(mentors_dedup_df)
#display(mentors_df)
display(mentors_dedup_1mentee_df)
display(mentors_dedup_2mentee_df)
display(mentors_dedup_3mentee_df)
